Here's an extensive summary in markdown format, drawing on the provided sources and our conversation history, with some bolding to highlight key concepts:

**Streaming Runnables in LangChain**

*   **Purpose**: This guide focuses on how to implement streaming in LangChain applications to make them feel more responsive to end-users. Streaming involves showing intermediate progress, such as output from an LLM token by token, rather than waiting for the entire response before displaying anything.

*   **Key Concepts**:
    *   **Runnable Interface**: Many LangChain primitives like chat models, output parsers, prompts, retrievers, and agents implement the LangChain Runnable Interface. This interface provides methods for streaming content.
    *  **Input Streams**: Streaming is possible when all steps in a program can process input as a stream, handling chunks of input one at a time and yielding corresponding output chunks.

*   **Streaming Methods**:
    *   **`stream` and `astream`**: These methods are designed to stream the final output in chunks. `stream` is synchronous, while `astream` is asynchronous.
    *  **`astream_events` and `astream_log`**: These asynchronous methods stream both intermediate steps and final output from the chain. The `astream_events` API is a beta feature, and may be subject to change.

*   **Streaming with LLMs and Chat Models**:
    *   LLMs are a primary bottleneck in LLM-based applications because they can take several seconds to generate a response, which is slower than the ~200-300 ms threshold for responsiveness.
    *   Streaming the output from the model token by token is key to making applications feel more responsive.
    *   The `stream` and `astream` methods yield `AIMessageChunk` objects, which represent a part of an `AIMessage`. These chunks can be added together to get the state of the response so far.

*   **Streaming with Chains**:
    *   Chains are constructed using the LangChain Expression Language (LCEL). LCEL chains automatically implement `stream` and `astream` for streaming the final output. LCEL also supports transform-style passthrough streaming, where primitives operate on each streaming chunk individually.
    *   Components like prompt templates and chat models may interrupt the streaming process because they cannot process individual chunks, instead aggregating all previous steps.
    *   Output parsers like `StrOutputParser` can be used to extract the content field from `AIMessageChunk` objects, giving the tokens returned by the model.
    *   Custom functions can be written as generators, which allow them to operate on streams.

*   **Working with Input Streams**:
    *   Parsers need to operate on the input stream in order to stream results, such as JSON, as they are being generated, by attempting to "auto-complete" the partial JSON into a valid state. For example, a `JsonOutputParser` can incrementally output JSON as it becomes available.

*   **Non-Streaming Components**:
    *   Some components, such as retrievers, do not support streaming.
    *   An LCEL chain that includes non-streaming components can still stream, with streaming of partial output beginning after the last non-streaming step.
    *   However, steps in a chain that operate on finalized inputs rather than input streams can break streaming functionality via `stream` or `astream`.

*   **Using Stream Events**:
    *   The `astream_events` API streams intermediate steps, even if a chain contains steps that only operate on finalized inputs.
    *   The API returns a variety of events from different components including `on_chat_model_start`, `on_chat_model_stream`, `on_chat_model_end`, `on_llm_start`, `on_llm_stream`, `on_llm_end`, `on_chain_start`, `on_chain_stream`, `on_chain_end`, `on_tool_start`, `on_tool_end`, `on_retriever_start`, `on_retriever_end`, `on_prompt_start`, and `on_prompt_end`.
    *   Events include the event name, the data associated with the event (such as chunk, input, or output), the name of the component that emitted the event, any tags associated with the component and a run ID. The input to a runnable may not be known until after the input stream has been fully consumed.
    *   Event streams can be filtered by component name, component type, or component tags.
    *   When using tools or custom runnables, callbacks need to be propagated correctly to generate stream events.  Callbacks are passed automatically by `RunnableLambdas` or the `@chain` decorator.

*   **Key Takeaways**
    *   Streaming is a crucial technique for making LLM-based applications more responsive by showing intermediate progress.
    *   LangChain's `Runnable` interface enables streaming through the `stream` and `astream` methods for final output, and the `astream_events` method for intermediate and final outputs.
    *   LCEL facilitates declarative specification of chains with automatic streaming implementation.
    *   Care must be taken to ensure that all steps in a chain can process input as a stream to avoid breaking the streaming functionality.
    *   The `astream_events` API provides granular control over streaming outputs by emitting events from all the steps in a chain, and can be filtered by name, type or tag.

This summary provides an overview of how to implement streaming in LangChain, drawing on the information provided in the sources and our conversation history, covering key concepts, methods, and options for constructing streaming applications.


In [2]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

In [3]:
chunks = []
for chunk in model.stream("what color is the sky?"):
    chunks.append(chunk)
    print(chunk.content, end="|", flush=True)

|The| color| of| the| sky| can| vary| depending| on| several| factors|,| including| the| time| of| day|,| weather| conditions|,| and| atmospheric| particles|.| During| a| clear| day|,| the| sky| typically| appears| blue| due| to| the| scattering| of| sunlight| by| the| atmosphere|.| Near| sunrise| and| sunset|,| the| sky| can| take| on| shades| of| orange|,| pink|,| and| purple|.| When| it's| cloudy| or| over|cast|,| the| sky| may| look| gray|.| Additionally|,| pollution| and| other| atmospheric| conditions| can| affect| the| sky|'s| color|.||

In [4]:
from langchain_ollama import ChatOllama

ollama = ChatOllama(model='llama3.1')
chunks = []
for chunk in ollama.stream("what color is the sky?"):
    chunks.append(chunk)
    print(chunk.content, end="|", flush=True)

The| answer| can| be| a| bit| tricky|,| as| it| depends| on| various| factors| such| as|:

|1|.| **|Time| of| day|**:| During| the| daytime| (|around| noon|),| the| sky| appears| blue| due| to| a| phenomenon| called| Ray|leigh| scattering|,| where| shorter| wavelengths| like| blue| light| are| scattered| more| than| longer| wavelengths| like| red| light| by| the| tiny| molecules| of| gases| in| the| atmosphere|.
|2|.| **|Weather| conditions|**:| On| cloudy| or| fog|gy| days|,| the| sky| can| appear| gray| or| white|.| During| sunrise| and| sunset|,| the| sky| takes| on| hues| of| orange|,| pink|,| and| purple| due| to| the| scattering| of| light| by| atmospheric| particles|.
|3|.| **|Location| and| altitude|**:| The| color| of| the| sky| can| change| depending| on| your| location|'s| latitude|,| altitude|,| and| proximity| to| urban| areas| or| pollution| sources|.

|So|,| to| answer| your| question|:| "|The| color| of| the| sky| is|...| blue| (|during| daytime|),| but| it| changes| to

In [5]:
chunks = []
async for chunk in model.astream("what color is the sky?"):
    chunks.append(chunk)
    print(chunk.content, end="|", flush=True)



|The| color| of| the| sky| can| vary| depending| on| the| time| of| day|,| weather| conditions|,| and| atmospheric| factors|.| During| a| clear| day|,| the| sky| typically| appears| blue| due| to| the| scattering| of| sunlight| by| the| atmosphere|.| At| sunrise| and| sunset|,| the| sky| can| take| on| shades| of| orange|,| pink|,| and| red|.| When| it's| cloudy| or| over|cast|,| the| sky| may| appear| gray|.| At| night|,| the| sky| is| usually| dark|,| often| dotted| with| stars|.||

In [6]:
chunks = []
async for chunk in ollama.astream("what color is the sky?"):
    chunks.append(chunk)
    print(chunk.content, end="|", flush=True)



The| answer|,| of|

 course|,| depends| on| the| time| of| day| and| atmospheric| conditions|.

|**|During| the| daytime|:|**
|When| the| sun| is| overhead|,| the| sky| appears| blue| because| of| a| phenomenon| called| Ray|leigh| scattering|.| Short|er| (|blue|)| wavelengths| of| light| are| scattered| more| than| longer| (|red|)| wavelengths| by| the| tiny| molecules| of| gases| in| the| atmosphere|,| making| the| sky| appear| blue|.

|**|At| sunrise| and| sunset|:|**
|As| the| sun| rises| or| sets|,| the| sky| can| take| on| hues| of| red|,| orange|,| pink|,| and| purple| due| to| a| combination| of| atmospheric| scattering| and| dust| particles|.| This| is| because| the| light| has| to| travel| through| more| of| the| Earth|'s| atmosphere| to| reach| our| eyes|,| which| sc|atters| shorter| wavelengths| even| further|.

|**|At| night|:|**
|The| sky| appears| dark| or| black|,| but| with| some| exceptions|:

|*| **|Moon|light|:**| The| full| moon| can| make| the| sky| appear| a| bright| gray|ish|-white|

In [7]:
chunks[0]



AIMessageChunk(content='The', additional_kwargs={}, response_metadata={}, id='run-ec786023-8e4a-49cb-b010-9e13d8992188')

In [8]:
chunks[0] + chunks[1] + chunks[2] + chunks[3] + chunks[4]



AIMessageChunk(content='The answer, of course', additional_kwargs={}, response_metadata={}, id='run-ec786023-8e4a-49cb-b010-9e13d8992188')

In [9]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
parser = StrOutputParser()
chain = prompt | model | parser

async for chunk in chain.astream({"topic": "parrot"}):
    print(chunk, end="|", flush=True)



|Why| did| the| par|rot| wear| a| rain|coat|?

|Because| it| wanted| to| be| a| poly|uns|aturated|!||

In [10]:
chain = prompt | ollama | parser

async for chunk in chain.astream({"topic": "parrot"}):
    print(chunk, end="|", flush=True)

Here|'s| one|:

|Why| did| the|

 par|rot| go| to| the| doctor|?

|Because| it| had| a| f|owl| cough|!| (|get| it|?)||

In [11]:
# the parser needs to operate on the input stream, and attempt to "auto-complete" the partial json into a valid state.

from langchain_core.output_parsers import JsonOutputParser

chain = (
    model | JsonOutputParser()
)  # Due to a bug in older versions of Langchain, JsonOutputParser did not stream results from some models
async for text in chain.astream(
    "output a list of the countries france, spain and japan and their populations in JSON format. "
    'Use a dict with an outer key of "countries" which contains a list of countries. '
    "Each country should have the key `name` and `population`"
):
    print(text, flush=True)

{}
{'countries': []}
{'countries': [{}]}
{'countries': [{'name': ''}]}
{'countries': [{'name': 'France'}]}
{'countries': [{'name': 'France', 'population': 652}]}
{'countries': [{'name': 'France', 'population': 652735}]}
{'countries': [{'name': 'France', 'population': 65273511}]}
{'countries': [{'name': 'France', 'population': 65273511}, {}]}
{'countries': [{'name': 'France', 'population': 65273511}, {'name': ''}]}
{'countries': [{'name': 'France', 'population': 65273511}, {'name': 'Spain'}]}
{'countries': [{'name': 'France', 'population': 65273511}, {'name': 'Spain', 'population': 467}]}
{'countries': [{'name': 'France', 'population': 65273511}, {'name': 'Spain', 'population': 467547}]}
{'countries': [{'name': 'France', 'population': 65273511}, {'name': 'Spain', 'population': 46754778}]}
{'countries': [{'name': 'France', 'population': 65273511}, {'name': 'Spain', 'population': 46754778}, {}]}
{'countries': [{'name': 'France', 'population': 65273511}, {'name': 'Spain', 'population': 467

In [12]:
from langchain_core.output_parsers import JsonOutputParser

chain = (
    ollama | JsonOutputParser()
)  # Due to a bug in older versions of Langchain, JsonOutputParser did not stream results from some models
async for text in chain.astream(
    "output a list of the countries france, spain and japan and their populations in JSON format. "
    'Use a dict with an outer key of "countries" which contains a list of countries. '
    "Each country should have the key `name` and `population`"
):
    print(text, flush=True)

{}
{'countries': []}
{'countries': [{}]}
{'countries': [{'name': ''}]}
{'countries': [{'name': 'France'}]}
{'countries': [{'name': 'France', 'population': 673}]}
{'countries': [{'name': 'France', 'population': 673261}]}
{'countries': [{'name': 'France', 'population': 67326100}]}
{'countries': [{'name': 'France', 'population': 67326100}, {}]}
{'countries': [{'name': 'France', 'population': 67326100}, {'name': ''}]}
{'countries': [{'name': 'France', 'population': 67326100}, {'name': 'Spain'}]}
{'countries': [{'name': 'France', 'population': 67326100}, {'name': 'Spain', 'population': 463}]}
{'countries': [{'name': 'France', 'population': 67326100}, {'name': 'Spain', 'population': 463331}]}
{'countries': [{'name': 'France', 'population': 67326100}, {'name': 'Spain', 'population': 46333111}]}
{'countries': [{'name': 'France', 'population': 67326100}, {'name': 'Spain', 'population': 46333111}, {}]}
{'countries': [{'name': 'France', 'population': 67326100}, {'name': 'Spain', 'population': 463

In [13]:
from langchain_core.output_parsers import (
    JsonOutputParser,
)


# A function that operates on finalized inputs
# rather than on an input_stream
def _extract_country_names(inputs):
    """A function that does not operates on input streams and breaks streaming."""
    if not isinstance(inputs, dict):
        return ""

    if "countries" not in inputs:
        return ""

    countries = inputs["countries"]

    if not isinstance(countries, list):
        return ""

    country_names = [
        country.get("name") for country in countries if isinstance(country, dict)
    ]
    return country_names


chain = model | JsonOutputParser() | _extract_country_names

async for text in chain.astream(
    "output a list of the countries france, spain and japan and their populations in JSON format. "
    'Use a dict with an outer key of "countries" which contains a list of countries. '
    "Each country should have the key `name` and `population`"
):
    print(text, end="|", flush=True)

['France', 'Spain', 'Japan']|

In [14]:
from langchain_core.output_parsers import JsonOutputParser


async def _extract_country_names_streaming(input_stream):
    """A function that operates on input streams."""
    country_names_so_far = set()

    async for input in input_stream:
        if not isinstance(input, dict):
            continue

        if "countries" not in input:
            continue

        countries = input["countries"]

        if not isinstance(countries, list):
            continue

        for country in countries:
            name = country.get("name")
            if not name:
                continue
            if name not in country_names_so_far:
                yield name
                country_names_so_far.add(name)


chain = model | JsonOutputParser() | _extract_country_names_streaming

async for text in chain.astream(
    "output a list of the countries france, spain and japan and their populations in JSON format. "
    'Use a dict with an outer key of "countries" which contains a list of countries. '
    "Each country should have the key `name` and `population`",
):
    print(text, end="|", flush=True)

France|Spain|Japan|

In [15]:
from langchain_core.output_parsers import JsonOutputParser


def _extract_country_names_streaming(input_stream):
    """A function that operates on input streams."""
    country_names_so_far = set()

    for input in input_stream:
        if not isinstance(input, dict):
            continue

        if "countries" not in input:
            continue

        countries = input["countries"]

        if not isinstance(countries, list):
            continue

        for country in countries:
            name = country.get("name")
            if not name:
                continue
            if name not in country_names_so_far:
                yield name
                country_names_so_far.add(name)


chain = model | JsonOutputParser() | _extract_country_names_streaming

for text in chain.stream(
    "output a list of the countries france, spain and japan and their populations in JSON format. "
    'Use a dict with an outer key of "countries" which contains a list of countries. '
    "Each country should have the key `name` and `population`",
):
    print(text, end="|", flush=True)

France|Spain|Japan|

In [16]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho", "harrison likes spicy food"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

chunks = [chunk for chunk in retriever.stream("where did harrison work?")]
chunks

[[Document(id='f4cd12ea-80b4-4f6b-86cd-d869ffdf19f9', metadata={}, page_content='harrison worked at kensho'),
  Document(id='1635956a-6dd6-459b-bfe0-4e49076191c8', metadata={}, page_content='harrison likes spicy food')]]

In [17]:
from langchain_ollama import OllamaEmbeddings

ollama_emb = OllamaEmbeddings(model="snowflake-arctic-embed2")

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho", "harrison likes spicy food"],
    embedding=ollama_emb,
)
retriever = vectorstore.as_retriever(search_kwargs={'k': 3})

chunks = [chunk for chunk in retriever.stream("where did harrison work?")]
chunks

[[Document(id='d2762d49-6554-49bc-b93c-0c98e73b4791', metadata={}, page_content='harrison worked at kensho'),
  Document(id='cf7895f1-5442-4e07-93fd-e38cef944239', metadata={}, page_content='harrison likes spicy food')]]

In [18]:
retrieval_chain = (
    {
        "context": retriever.with_config(run_name="Docs"),
        "question": RunnablePassthrough(),
    }
    | prompt
    | model
    | StrOutputParser()
)

In [19]:
for chunk in retrieval_chain.stream(
    "Where did harrison work? " "Write 3 made up sentences about this place."
):
    print(chunk, end="|", flush=True)

|H|arrison| worked| at| Kens|ho|.| Kens|ho| is| known| for| its| innovative| approach| to| data| analysis|,| blending| cutting|-edge| technology| with| deep| market| insights|.| The| team| at| Kens|ho| often| collabor|ates| on| projects| that| require| both| creativity| and| analytical| rigor|,| creating| a| dynamic| work| environment|.| Employees| frequently| participate| in| brainstorming| sessions| that| encourage| out|-of|-the|-box| thinking|,| making| it| a| hub| for| forward|-thinking| professionals|.||

In [20]:
events = []
async for event in model.astream_events("hello", version="v2"):
    events.append(event)

In [22]:
events[:3]

[{'event': 'on_chat_model_start',
  'data': {'input': 'hello'},
  'name': 'ChatOpenAI',
  'tags': [],
  'run_id': '0858c65b-a131-4175-b174-53898ed99c6a',
  'metadata': {'ls_provider': 'openai',
   'ls_model_name': 'gpt-4o-mini',
   'ls_model_type': 'chat',
   'ls_temperature': 0.7},
  'parent_ids': []},
 {'event': 'on_chat_model_stream',
  'run_id': '0858c65b-a131-4175-b174-53898ed99c6a',
  'name': 'ChatOpenAI',
  'tags': [],
  'metadata': {'ls_provider': 'openai',
   'ls_model_name': 'gpt-4o-mini',
   'ls_model_type': 'chat',
   'ls_temperature': 0.7},
  'data': {'chunk': AIMessageChunk(content='', additional_kwargs={}, response_metadata={}, id='run-0858c65b-a131-4175-b174-53898ed99c6a')},
  'parent_ids': []},
 {'event': 'on_chat_model_stream',
  'run_id': '0858c65b-a131-4175-b174-53898ed99c6a',
  'name': 'ChatOpenAI',
  'tags': [],
  'metadata': {'ls_provider': 'openai',
   'ls_model_name': 'gpt-4o-mini',
   'ls_model_type': 'chat',
   'ls_temperature': 0.7},
  'data': {'chunk': AIMe

In [23]:
events[-2:]


[{'event': 'on_chat_model_stream',
  'run_id': '0858c65b-a131-4175-b174-53898ed99c6a',
  'name': 'ChatOpenAI',
  'tags': [],
  'metadata': {'ls_provider': 'openai',
   'ls_model_name': 'gpt-4o-mini',
   'ls_model_type': 'chat',
   'ls_temperature': 0.7},
  'data': {'chunk': AIMessageChunk(content='', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b'}, id='run-0858c65b-a131-4175-b174-53898ed99c6a')},
  'parent_ids': []},
 {'event': 'on_chat_model_end',
  'data': {'output': AIMessageChunk(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b'}, id='run-0858c65b-a131-4175-b174-53898ed99c6a')},
  'run_id': '0858c65b-a131-4175-b174-53898ed99c6a',
  'name': 'ChatOpenAI',
  'tags': [],
  'metadata': {'ls_provider': 'openai',
   'ls_model_name': 'gpt-4o-mini',
   'ls_m

In [24]:
chain = (
    model | JsonOutputParser()
)  # Due to a bug in older versions of Langchain, JsonOutputParser did not stream results from some models

events = [
    event
    async for event in chain.astream_events(
        "output a list of the countries france, spain and japan and their populations in JSON format. "
        'Use a dict with an outer key of "countries" which contains a list of countries. '
        "Each country should have the key `name` and `population`",
        version="v2",
    )
]

In [25]:
events[:3]


[{'event': 'on_chain_start',
  'data': {'input': 'output a list of the countries france, spain and japan and their populations in JSON format. Use a dict with an outer key of "countries" which contains a list of countries. Each country should have the key `name` and `population`'},
  'name': 'RunnableSequence',
  'tags': [],
  'run_id': 'f3a9d910-6526-49be-a81f-b77553425bf0',
  'metadata': {},
  'parent_ids': []},
 {'event': 'on_chat_model_start',
  'data': {'input': {'messages': [[HumanMessage(content='output a list of the countries france, spain and japan and their populations in JSON format. Use a dict with an outer key of "countries" which contains a list of countries. Each country should have the key `name` and `population`', additional_kwargs={}, response_metadata={})]]}},
  'name': 'ChatOpenAI',
  'tags': ['seq:step:1'],
  'run_id': '01fae1c4-6b42-4232-9f55-89fcf19fe86e',
  'metadata': {'ls_provider': 'openai',
   'ls_model_name': 'gpt-4o-mini',
   'ls_model_type': 'chat',
   'l

In [26]:
events[-2:]


[{'event': 'on_parser_end',
  'data': {'output': {'countries': [{'name': 'France', 'population': 65273511},
     {'name': 'Spain', 'population': 46754778},
     {'name': 'Japan', 'population': 126476461}]},
   'input': AIMessageChunk(content='Here is the JSON representation of the countries France, Spain, and Japan along with their populations:\n\n```json\n{\n  "countries": [\n    {\n      "name": "France",\n      "population": 65273511\n    },\n    {\n      "name": "Spain",\n      "population": 46754778\n    },\n    {\n      "name": "Japan",\n      "population": 126476461\n    }\n  ]\n}\n```\n\nPlease note that the population figures are based on estimates as of 2021, and actual numbers may vary.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b'}, id='run-01fae1c4-6b42-4232-9f55-89fcf19fe86e')},
  'run_id': 'ea3ec37e-e9e4-4bec-aa6a-743859a06451',
  'name': 'JsonOutputParser',
  'tags': ['se

In [27]:
num_events = 0

async for event in chain.astream_events(
    "output a list of the countries france, spain and japan and their populations in JSON format. "
    'Use a dict with an outer key of "countries" which contains a list of countries. '
    "Each country should have the key `name` and `population`",
    version="v2",
):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        print(
            f"Chat model chunk: {repr(event['data']['chunk'].content)}",
            flush=True,
        )
    if kind == "on_parser_stream":
        print(f"Parser chunk: {event['data']['chunk']}", flush=True)
    num_events += 1
    if num_events > 30:
        # Truncate the output
        print("...")
        break

Chat model chunk: ''
Chat model chunk: 'Here'
Chat model chunk: ' is'
Chat model chunk: ' the'
Chat model chunk: ' JSON'
Chat model chunk: ' representation'
Chat model chunk: ' of'
Chat model chunk: ' the'
Chat model chunk: ' countries'
Chat model chunk: ' France'
Chat model chunk: ','
Chat model chunk: ' Spain'
Chat model chunk: ','
Chat model chunk: ' and'
Chat model chunk: ' Japan'
Chat model chunk: ' along'
Chat model chunk: ' with'
Chat model chunk: ' their'
Chat model chunk: ' populations'
Chat model chunk: ':\n\n'
Chat model chunk: '```'
Chat model chunk: 'json'
Chat model chunk: '\n'
Chat model chunk: '{\n'
Parser chunk: {}
Chat model chunk: ' '
Chat model chunk: ' "'
...


In [28]:
chain = model.with_config({"run_name": "model"}) | JsonOutputParser().with_config(
    {"run_name": "my_parser"}
)

max_events = 0
async for event in chain.astream_events(
    "output a list of the countries france, spain and japan and their populations in JSON format. "
    'Use a dict with an outer key of "countries" which contains a list of countries. '
    "Each country should have the key `name` and `population`",
    version="v2",
    include_names=["my_parser"],
):
    print(event)
    max_events += 1
    if max_events > 10:
        # Truncate output
        print("...")
        break

{'event': 'on_parser_start', 'data': {'input': 'output a list of the countries france, spain and japan and their populations in JSON format. Use a dict with an outer key of "countries" which contains a list of countries. Each country should have the key `name` and `population`'}, 'name': 'my_parser', 'tags': ['seq:step:2'], 'run_id': '91df3da0-9174-4384-82ce-44e895661de8', 'metadata': {}, 'parent_ids': ['a6d5985d-5e6c-49c2-846d-b9ecf5b812a2']}
{'event': 'on_parser_stream', 'run_id': '91df3da0-9174-4384-82ce-44e895661de8', 'name': 'my_parser', 'tags': ['seq:step:2'], 'metadata': {}, 'data': {'chunk': {}}, 'parent_ids': ['a6d5985d-5e6c-49c2-846d-b9ecf5b812a2']}
{'event': 'on_parser_stream', 'run_id': '91df3da0-9174-4384-82ce-44e895661de8', 'name': 'my_parser', 'tags': ['seq:step:2'], 'metadata': {}, 'data': {'chunk': {'countries': []}}, 'parent_ids': ['a6d5985d-5e6c-49c2-846d-b9ecf5b812a2']}
{'event': 'on_parser_stream', 'run_id': '91df3da0-9174-4384-82ce-44e895661de8', 'name': 'my_parse

In [29]:
chain = model.with_config({"run_name": "model"}) | JsonOutputParser().with_config(
    {"run_name": "my_parser"}
)

max_events = 0
async for event in chain.astream_events(
    'output a list of the countries france, spain and japan and their populations in JSON format. Use a dict with an outer key of "countries" which contains a list of countries. Each country should have the key `name` and `population`',
    version="v2",
    include_types=["chat_model"],
):
    print(event)
    max_events += 1
    if max_events > 10:
        # Truncate output
        print("...")
        break

{'event': 'on_chat_model_start', 'data': {'input': 'output a list of the countries france, spain and japan and their populations in JSON format. Use a dict with an outer key of "countries" which contains a list of countries. Each country should have the key `name` and `population`'}, 'name': 'model', 'tags': ['seq:step:1'], 'run_id': '0dd36d4b-a3fd-409c-b69b-8c9f3bb82260', 'metadata': {'ls_provider': 'openai', 'ls_model_name': 'gpt-4o-mini', 'ls_model_type': 'chat', 'ls_temperature': 0.7}, 'parent_ids': ['05cc7f48-b5c9-463c-b9b9-a145ea51f811']}
{'event': 'on_chat_model_stream', 'data': {'chunk': AIMessageChunk(content='', additional_kwargs={}, response_metadata={}, id='run-0dd36d4b-a3fd-409c-b69b-8c9f3bb82260')}, 'run_id': '0dd36d4b-a3fd-409c-b69b-8c9f3bb82260', 'name': 'model', 'tags': ['seq:step:1'], 'metadata': {'ls_provider': 'openai', 'ls_model_name': 'gpt-4o-mini', 'ls_model_type': 'chat', 'ls_temperature': 0.7}, 'parent_ids': ['05cc7f48-b5c9-463c-b9b9-a145ea51f811']}
{'event': '

In [30]:
chain = (model | JsonOutputParser()).with_config({"tags": ["my_chain"]})

max_events = 0
async for event in chain.astream_events(
    'output a list of the countries france, spain and japan and their populations in JSON format. Use a dict with an outer key of "countries" which contains a list of countries. Each country should have the key `name` and `population`',
    version="v2",
    include_tags=["my_chain"],
):
    print(event)
    max_events += 1
    if max_events > 10:
        # Truncate output
        print("...")
        break

{'event': 'on_chain_start', 'data': {'input': 'output a list of the countries france, spain and japan and their populations in JSON format. Use a dict with an outer key of "countries" which contains a list of countries. Each country should have the key `name` and `population`'}, 'name': 'RunnableSequence', 'tags': ['my_chain'], 'run_id': '34742eb5-82bc-450c-b619-aa22d743c78c', 'metadata': {}, 'parent_ids': []}
{'event': 'on_chat_model_start', 'data': {'input': {'messages': [[HumanMessage(content='output a list of the countries france, spain and japan and their populations in JSON format. Use a dict with an outer key of "countries" which contains a list of countries. Each country should have the key `name` and `population`', additional_kwargs={}, response_metadata={})]]}}, 'name': 'ChatOpenAI', 'tags': ['seq:step:1', 'my_chain'], 'run_id': 'a0e02020-78e5-4fb0-823d-c2af9a2d1a56', 'metadata': {'ls_provider': 'openai', 'ls_model_name': 'gpt-4o-mini', 'ls_model_type': 'chat', 'ls_temperatur

In [31]:
# Function that does not support streaming.
# It operates on the finalizes inputs rather than
# operating on the input stream.
def _extract_country_names(inputs):
    """A function that does not operates on input streams and breaks streaming."""
    if not isinstance(inputs, dict):
        return ""

    if "countries" not in inputs:
        return ""

    countries = inputs["countries"]

    if not isinstance(countries, list):
        return ""

    country_names = [
        country.get("name") for country in countries if isinstance(country, dict)
    ]
    return country_names


chain = (
    model | JsonOutputParser() | _extract_country_names
)  # This parser only works with OpenAI right now



In [32]:
async for chunk in chain.astream(
    "output a list of the countries france, spain and japan and their populations in JSON format. "
    'Use a dict with an outer key of "countries" which contains a list of countries. '
    "Each country should have the key `name` and `population`",
):
    print(chunk, flush=True)

['France', 'Spain', 'Japan']


In [33]:
num_events = 0

async for event in chain.astream_events(
    "output a list of the countries france, spain and japan and their populations in JSON format. "
    'Use a dict with an outer key of "countries" which contains a list of countries. '
    "Each country should have the key `name` and `population`",
    version="v2",
):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        print(
            f"Chat model chunk: {repr(event['data']['chunk'].content)}",
            flush=True,
        )
    if kind == "on_parser_stream":
        print(f"Parser chunk: {event['data']['chunk']}", flush=True)
    num_events += 1
    if num_events > 30:
        # Truncate the output
        print("...")
        break

Chat model chunk: ''
Chat model chunk: 'Here'
Chat model chunk: ' is'
Chat model chunk: ' the'
Chat model chunk: ' JSON'
Chat model chunk: ' representation'
Chat model chunk: ' of'
Chat model chunk: ' the'
Chat model chunk: ' countries'
Chat model chunk: ' France'
Chat model chunk: ','
Chat model chunk: ' Spain'
Chat model chunk: ','
Chat model chunk: ' and'
Chat model chunk: ' Japan'
Chat model chunk: ' along'
Chat model chunk: ' with'
Chat model chunk: ' their'
Chat model chunk: ' populations'
Chat model chunk: ':\n\n'
Chat model chunk: '```'
Chat model chunk: 'json'
Chat model chunk: '\n'
Chat model chunk: '{\n'
Parser chunk: {}
Chat model chunk: ' '
Chat model chunk: ' "'
...


In [34]:
from langchain_core.runnables import RunnableLambda
from langchain_core.tools import tool


def reverse_word(word: str):
    return word[::-1]


reverse_word = RunnableLambda(reverse_word)


@tool
def bad_tool(word: str):
    """Custom tool that doesn't propagate callbacks."""
    return reverse_word.invoke(word)


async for event in bad_tool.astream_events("hello", version="v2"):
    print(event)

{'event': 'on_tool_start', 'data': {'input': 'hello'}, 'name': 'bad_tool', 'tags': [], 'run_id': 'a2c37a3b-49f8-41bc-a1ac-692b7d4126dd', 'metadata': {}, 'parent_ids': []}
{'event': 'on_chain_start', 'data': {'input': 'hello'}, 'name': 'reverse_word', 'tags': [], 'run_id': 'fb9b419a-8333-4077-907a-9efd327a295d', 'metadata': {}, 'parent_ids': ['a2c37a3b-49f8-41bc-a1ac-692b7d4126dd']}
{'event': 'on_chain_end', 'data': {'output': 'olleh', 'input': 'hello'}, 'run_id': 'fb9b419a-8333-4077-907a-9efd327a295d', 'name': 'reverse_word', 'tags': [], 'metadata': {}, 'parent_ids': ['a2c37a3b-49f8-41bc-a1ac-692b7d4126dd']}
{'event': 'on_tool_end', 'data': {'output': 'olleh'}, 'run_id': 'a2c37a3b-49f8-41bc-a1ac-692b7d4126dd', 'name': 'bad_tool', 'tags': [], 'metadata': {}, 'parent_ids': []}


In [35]:
@tool
def correct_tool(word: str, callbacks):
    """A tool that correctly propagates callbacks."""
    return reverse_word.invoke(word, {"callbacks": callbacks})


async for event in correct_tool.astream_events("hello", version="v2"):
    print(event)

{'event': 'on_tool_start', 'data': {'input': 'hello'}, 'name': 'correct_tool', 'tags': [], 'run_id': '17425d62-e534-46a2-91f9-374a4beb5125', 'metadata': {}, 'parent_ids': []}
{'event': 'on_chain_start', 'data': {'input': 'hello'}, 'name': 'reverse_word', 'tags': [], 'run_id': '4a7ce7f2-b5f4-4881-9df9-f80b776d9076', 'metadata': {}, 'parent_ids': ['17425d62-e534-46a2-91f9-374a4beb5125']}
{'event': 'on_chain_end', 'data': {'output': 'olleh', 'input': 'hello'}, 'run_id': '4a7ce7f2-b5f4-4881-9df9-f80b776d9076', 'name': 'reverse_word', 'tags': [], 'metadata': {}, 'parent_ids': ['17425d62-e534-46a2-91f9-374a4beb5125']}
{'event': 'on_tool_end', 'data': {'output': 'olleh'}, 'run_id': '17425d62-e534-46a2-91f9-374a4beb5125', 'name': 'correct_tool', 'tags': [], 'metadata': {}, 'parent_ids': []}


In [36]:
from langchain_core.runnables import RunnableLambda


async def reverse_and_double(word: str):
    return await reverse_word.ainvoke(word) * 2


reverse_and_double = RunnableLambda(reverse_and_double)

await reverse_and_double.ainvoke("1234")

async for event in reverse_and_double.astream_events("1234", version="v2"):
    print(event)

{'event': 'on_chain_start', 'data': {'input': '1234'}, 'name': 'reverse_and_double', 'tags': [], 'run_id': 'e14b3302-9b61-4e5d-abfb-a350c077955d', 'metadata': {}, 'parent_ids': []}
{'event': 'on_chain_start', 'data': {'input': '1234'}, 'name': 'reverse_word', 'tags': [], 'run_id': 'c5e41cbf-e127-4dab-b3bb-d8a501ce5f68', 'metadata': {}, 'parent_ids': ['e14b3302-9b61-4e5d-abfb-a350c077955d']}
{'event': 'on_chain_end', 'data': {'output': '4321', 'input': '1234'}, 'run_id': 'c5e41cbf-e127-4dab-b3bb-d8a501ce5f68', 'name': 'reverse_word', 'tags': [], 'metadata': {}, 'parent_ids': ['e14b3302-9b61-4e5d-abfb-a350c077955d']}
{'event': 'on_chain_stream', 'run_id': 'e14b3302-9b61-4e5d-abfb-a350c077955d', 'name': 'reverse_and_double', 'tags': [], 'metadata': {}, 'data': {'chunk': '43214321'}, 'parent_ids': []}
{'event': 'on_chain_end', 'data': {'output': '43214321'}, 'run_id': 'e14b3302-9b61-4e5d-abfb-a350c077955d', 'name': 'reverse_and_double', 'tags': [], 'metadata': {}, 'parent_ids': []}


In [37]:
from langchain_core.runnables import chain


@chain
async def reverse_and_double(word: str):
    return await reverse_word.ainvoke(word) * 2


await reverse_and_double.ainvoke("1234")

async for event in reverse_and_double.astream_events("1234", version="v2"):
    print(event)

{'event': 'on_chain_start', 'data': {'input': '1234'}, 'name': 'reverse_and_double', 'tags': [], 'run_id': 'c46bc257-6003-4b5a-9ba3-06e17908b3ac', 'metadata': {}, 'parent_ids': []}
{'event': 'on_chain_start', 'data': {'input': '1234'}, 'name': 'reverse_word', 'tags': [], 'run_id': 'dd021df8-2b1d-4d15-baec-ae6cd825eca3', 'metadata': {}, 'parent_ids': ['c46bc257-6003-4b5a-9ba3-06e17908b3ac']}
{'event': 'on_chain_end', 'data': {'output': '4321', 'input': '1234'}, 'run_id': 'dd021df8-2b1d-4d15-baec-ae6cd825eca3', 'name': 'reverse_word', 'tags': [], 'metadata': {}, 'parent_ids': ['c46bc257-6003-4b5a-9ba3-06e17908b3ac']}
{'event': 'on_chain_stream', 'run_id': 'c46bc257-6003-4b5a-9ba3-06e17908b3ac', 'name': 'reverse_and_double', 'tags': [], 'metadata': {}, 'data': {'chunk': '43214321'}, 'parent_ids': []}
{'event': 'on_chain_end', 'data': {'output': '43214321'}, 'run_id': 'c46bc257-6003-4b5a-9ba3-06e17908b3ac', 'name': 'reverse_and_double', 'tags': [], 'metadata': {}, 'parent_ids': []}
